## plotly

In [9]:
import plotly.graph_objects as go
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 计算 Bollinger Bands (20 日均线，标准差 2)
def plot_kline(df_resampled):
    window = 20
    kbars["SMA"] = kbars["close"].rolling(5).mean()
    kbars["LMA"] = kbars["close"].rolling(window).mean()
    kbars["std"] = df_resampled["close"].rolling(window).std()
    df_resampled["upper"] = df_resampled["LMA"] + 2 * df_resampled["std"]
    df_resampled["lower"] = df_resampled["SMA"] - 2 * df_resampled["std"]

    # 计算最大成交量，并设定右轴范围（5 倍最大成交量）
    max_volume = df_resampled["volume"].max()
    scaled_volume = max_volume * 8
    magnitude = 8 ** (len(str(int(scaled_volume))) - 1)
    adjusted_max_volume = ((scaled_volume // magnitude) + 1) * magnitude

    # 使用 Plotly 绘制 K 线图与成交量
    fig = go.Figure()

    # 添加 K 线图
    fig.add_trace(go.Candlestick(
        x=df_resampled.index,
        open=df_resampled["open"],
        high=df_resampled["high"],
        low=df_resampled["low"],
        close=df_resampled["close"],
        name="Candlestick"
    ))

    fig.add_trace(go.Scatter(
        x=df_resampled.index,
        y=df_resampled["SMA"],
        name="SMA",
        line=dict(color="red", width=0.5),
        mode="lines"
    ))


    # 添加 Bollinger Bands 中
    fig.add_trace(go.Scatter(
        x=df_resampled.index,
        y=df_resampled["LMA"],
        name="LMA",
        line=dict(color="gray", width=0.5),
        mode="lines"
    ))

    # 添加 Bollinger Bands 上轨
    fig.add_trace(go.Scatter(
        x=df_resampled.index,
        y=df_resampled["upper"],
        name="Upper Band",
        line=dict(color="orange", width=1),
        mode="lines"
    ))

    # 添加 Bollinger Bands 下轨
    fig.add_trace(go.Scatter(
        x=df_resampled.index,
        y=df_resampled["lower"],
        name="Lower Band",
        line=dict(color="orange", width=1),
        mode="lines",
        fill="tonexty",  # 填充上下轨之间的区域
        fillcolor="rgba(255,165,0,0.2)"  # 半透明橙色
    ))

    # 添加成交量图
    fig.add_trace(go.Bar(
        x=df_resampled.index,
        y=df_resampled["volume"],
        name="Volume",
        marker=dict(color="blue"),
        yaxis="y2"
    ))

    # 更新图表布局
    fig.update_layout(
        title="Kbars with Bollinger Bands & Volume",
        xaxis_title="Time",
        yaxis=dict(title="Price"),
        yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False, range=[0, adjusted_max_volume]),
        width=1200,
        height=600
    )

    fig.show()


## seed

In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def simu_price(seed = 21,cnt = 200_000):
    # date_range = pd.date_range(start="2023-01-02 09:30:00", periods=1000000, freq="1min")
    start_date = "2022-01-02"
    total_minutes_per_day = (15 - 9) * 60  # 9:00 - 15:00 共有 360 分钟
    days_needed = cnt // total_minutes_per_day  # 计算需要多少个交易日

    # 生成符合交易时间的日期（周一到周五）
    all_dates = pd.date_range(start=start_date, periods=days_needed * 2, freq="B")[:days_needed]  # 确保取足够交易日

    # 生成每日交易时间（9:00 - 15:00，每分钟）
    trading_minutes = pd.date_range("09:00", "15:00", freq="1min").time

    # 组合日期和交易分钟生成完整的时间序列
    date_range = [pd.Timestamp(date).replace(hour=t.hour, minute=t.minute) for date in all_dates for t in trading_minutes]
    date_range = pd.DatetimeIndex(date_range)[:cnt]  # 截取接近100万条数据
    np.random.seed(seed)
    price_changes = np.random.uniform(-0.03, 0.03, cnt)
    price_changes *= 0.0003 / np.std(price_changes)  # 调整方差

    # 处理连续3次同方向变化的情况
    for i in range(2, len(price_changes)):
        if price_changes[i-4] > 0 and price_changes[i-3] > 0 and price_changes[i-2] > 0 and price_changes[i-1] > 0 and price_changes[i] > 0:
            price_changes[i] = -abs(price_changes[i])
        elif price_changes[i-4] < 0 and price_changes[i-3] < 0 and price_changes[i-2] < 0 and price_changes[i-1] < 0 and price_changes[i] < 0:
            price_changes[i] = abs(price_changes[i])

    # 计算价格
    prices = [100]
    for change in price_changes:
        prices.append(round(prices[-1] * (1 + change),2))
    prices = prices[1:]

    # 计算high, low, close
    df = pd.DataFrame({
        "datetime": date_range,
        "open": prices,
    })
    df["high"] = df["open"].rolling(5).max()
    df["low"] = df["open"].rolling(5).min()
    df["close"] = df["open"].shift(-1)
    df.fillna(method='ffill', inplace=True)
    kbars = df.resample("1d", on="datetime").agg({
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last"
    }).dropna()

    vol = kbars["high"] - kbars["low"]
    vol_std = vol.rolling(5).std().fillna(0)
    kbars["volume"] = np.random.randint(800, 1200, len(kbars)) * (1 + np.exp(vol_std * 5))
    return kbars

In [30]:
kbars = simu_price(seed = 21,cnt = 300_000)
plot_kline(kbars)

# Backtesting

In [ ]:
import pandas as pd
import numpy as np

# 假設 kbars 已經包含時間序列數據
window_short = 5
window_long = 20
initial_capital = 1_000_000
min_trade_unit = 100

# 計算均線
kbars["SMA"] = kbars["close"].rolling(window_short).mean()
kbars["LMA"] = kbars["close"].rolling(window_long).mean()

# 交易信號
# 1: 買入, -1: 賣出, 0: 無動作
kbars["signal"] = np.where(kbars["SMA"] > kbars["LMA"], 1, 0)
kbars["signal"] = np.where(kbars["SMA"] < kbars["LMA"], -1, kbars["signal"])

def backtest(df, initial_capital, min_trade_unit):
    capital = initial_capital
    position = 0
    peak = initial_capital
    records = []
    
    for index, row in df.iterrows():
        if row.signal == 1 and capital > 0:  # 買入
            shares = (capital // row.close) // min_trade_unit * min_trade_unit
            capital -= shares * row.close
            position += shares
        elif row.signal == -1 and position > 0:  # 賣出
            capital += position * row.close
            position = 0
        
        total_value = capital + position * row.close
        peak = max(peak, total_value)
        drawdown = (peak - total_value) / peak
        
        records.append({
            "timestamp": index,
            "close": row.close,
            "SMA": row.SMA,
            "LMA": row.LMA,
            "signal": row.signal,
            "capital": capital,
            "shares": position,
            "equity": total_value,
            "drawdown": drawdown
        })
    
    return pd.DataFrame(records).set_index("timestamp")

df_backtest = backtest(kbars, initial_capital, min_trade_unit)

# 最終盈虧與最大回撤
final_profit = df_backtest["equity"].iloc[-1] - initial_capital
max_drawdown = df_backtest["drawdown"].max()

print(f"最終盈虧: {final_profit:.2f}")
print(f"最大回撤: {max_drawdown:.2%}")

最終盈虧: 47187.00
最大回撤: 7.11%


,close,SMA,LMA,signal,capital,shares,equity,drawdown
timestamp,,,,,,,,
2022-01-28,101.94,101.250,100.5775,1.0,988.0,9800.0,1000000.0,0.000000
2022-01-31,101.49,101.426,100.6435,1.0,988.0,9800.0,995590.0,0.004410
2022-02-01,102.01,101.618,100.7245,1.0,988.0,9800.0,1000686.0,0.000000
2022-02-02,102.06,101.824,100.8045,1.0,988.0,9800.0,1001176.0,0.000000
2022-02-03,103.06,102.112,100.9395,1.0,988.0,9800.0,1010976.0,0.000000
...,...,...,...,...,...,...,...,...
2025-03-03,89.02,88.682,87.9270,1.0,7267.0,12000.0,1075507.0,0.029864
2025-03-04,88.72,88.832,87.9810,1.0,7267.0,12000.0,1071907.0,0.033112
2025-03-05,88.54,88.818,88.0430,1.0,7267.0,12000.0,1069747.0,0.035060
